# 05 - Modelo Descriptivo: Regresión Mensual de Delitos

## Descripción del Dataset
Dataset mensual con **60 variables** incluyendo lags, rolling windows, estacionalidad y tasas por municipio-año-mes.

## Objetivo del Notebook
Generar **estadísticas pre-calculadas** en formato JSON para uso inmediato en:
- **Tableros de visualización** (dashboards)
- **Chatbots** con respuestas instantáneas
- **APIs** de consulta rápida

## Archivos Generados
| Archivo | Contenido |
|---------|-----------|
| `estadisticas_generales.json` | Métricas globales del dataset |
| `estadisticas_por_municipio.json` | Análisis detallado por municipio |
| `tendencias_temporales.json` | Patrones por año y mes (estacionalidad) |
| `comparativas.json` | Rankings y comparaciones entre municipios |
| `respuestas_chatbot.json` | Respuestas pre-generadas para preguntas frecuentes |

## Preguntas que Responde
- ¿Cuántos delitos hubo en promedio el último año?
- ¿Cuál es el municipio con más/menos delitos?
- ¿Qué mes tiene mayor incidencia delictiva?
- ¿Cómo evolucionó la criminalidad por año?
- ¿Qué municipios tuvieron mayor incremento o reducción?

## Utilidad Práctica
- **Respuestas instantáneas** sin procesar datos en tiempo real
- **Contexto histórico** para interpretar predicciones del modelo predictivo
- **Rankings actualizados** para priorización de recursos policiales

In [1]:
import pandas as pd
import numpy as np
import json
from pathlib import Path
from datetime import datetime

# Rutas
BASE_DIR = Path().resolve().parent
DATA_PATH = BASE_DIR / 'data' / 'gold' / 'model' / 'regression_monthly_dataset.parquet'
OUTPUT_DIR = BASE_DIR / 'models' / 'descriptivo' / 'regression_monthly'
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

print(f"📁 Datos: {DATA_PATH}")
print(f"📁 Salida: {OUTPUT_DIR}")

📁 Datos: /Users/byverbel/quick_projects/Datos-al-Ecosistema/data/gold/model/regression_monthly_dataset.parquet
📁 Salida: /Users/byverbel/quick_projects/Datos-al-Ecosistema/models/descriptivo/regression_monthly


In [2]:
# Cargar datos
df = pd.read_parquet(DATA_PATH)
print(f" Dataset cargado: {df.shape[0]:,} registros, {df.shape[1]} columnas")
print(f" Período: {df['anio'].min()} - {df['anio'].max()}")
print(f" Municipios: {df['codigo_municipio'].nunique()}")

 Dataset cargado: 13,044 registros, 60 columnas
 Período: 2010 - 2025
 Municipios: 87


---
## 1. Estadísticas Generales

In [3]:
# 1.1 Calcular estadísticas generales
print(" CALCULANDO ESTADÍSTICAS GENERALES")
print("=" * 60)

target = 'total_delitos'

# Último año completo
ultimo_anio = df['anio'].max()
anio_anterior = ultimo_anio - 1

# Estadísticas por período
df_ultimo = df[df['anio'] == ultimo_anio]
df_anterior = df[df['anio'] == anio_anterior]

estadisticas_generales = {
    'fecha_generacion': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    'dataset': 'regression_monthly_dataset.parquet',
    'periodo': {
        'inicio': int(df['anio'].min()),
        'fin': int(df['anio'].max()),
        'total_meses': int(df.groupby(['anio', 'mes']).ngroups),
        'ultimo_anio_completo': int(ultimo_anio)
    },
    'cobertura': {
        'total_registros': int(len(df)),
        'municipios': int(df['codigo_municipio'].nunique()),
        'registros_por_municipio': round(len(df) / df['codigo_municipio'].nunique(), 1)
    },
    'delitos_historico': {
        'total_acumulado': int(df[target].sum()),
        'promedio_mensual_global': round(df[target].mean(), 2),
        'mediana_mensual': round(df[target].median(), 2),
        'desviacion_estandar': round(df[target].std(), 2),
        'minimo_mensual': int(df[target].min()),
        'maximo_mensual': int(df[target].max())
    },
    'delitos_ultimo_anio': {
        'total': int(df_ultimo[target].sum()),
        'promedio_mensual': round(df_ultimo[target].mean(), 2),
        'variacion_vs_anterior': round(
            (df_ultimo[target].sum() - df_anterior[target].sum()) / df_anterior[target].sum() * 100, 2
        ) if df_anterior[target].sum() > 0 else 0
    }
}

print(json.dumps(estadisticas_generales, indent=2, ensure_ascii=False))

 CALCULANDO ESTADÍSTICAS GENERALES
{
  "fecha_generacion": "2025-11-28 22:16:52",
  "dataset": "regression_monthly_dataset.parquet",
  "periodo": {
    "inicio": 2010,
    "fin": 2025,
    "total_meses": 190,
    "ultimo_anio_completo": 2025
  },
  "cobertura": {
    "total_registros": 13044,
    "municipios": 87,
    "registros_por_municipio": 149.9
  },
  "delitos_historico": {
    "total_acumulado": 508227,
    "promedio_mensual_global": 38.96,
    "mediana_mensual": 4.0,
    "desviacion_estandar": 141.52,
    "minimo_mensual": 1,
    "maximo_mensual": 1723
  },
  "delitos_ultimo_anio": {
    "total": 39248,
    "promedio_mensual": 57.05,
    "variacion_vs_anterior": 21.27
  }
}


---
## 2. Estadísticas por Municipio

In [4]:
# 2.1 Estadísticas detalladas por municipio
print(" CALCULANDO ESTADÍSTICAS POR MUNICIPIO")
print("=" * 60)

estadisticas_municipio = {}

for cod_mun in df['codigo_municipio'].unique():
    df_mun = df[df['codigo_municipio'] == cod_mun]
    
    # Último año vs anterior
    df_mun_ultimo = df_mun[df_mun['anio'] == ultimo_anio]
    df_mun_anterior = df_mun[df_mun['anio'] == anio_anterior]
    
    total_ultimo = df_mun_ultimo[target].sum()
    total_anterior = df_mun_anterior[target].sum()
    
    if total_anterior > 0:
        variacion_pct = round((total_ultimo - total_anterior) / total_anterior * 100, 2)
    else:
        variacion_pct = 0
    
    # Tendencia (regresión lineal simple sobre promedios anuales)
    promedios_anuales = df_mun.groupby('anio')[target].mean()
    if len(promedios_anuales) >= 2:
        x = np.arange(len(promedios_anuales))
        slope = np.polyfit(x, promedios_anuales.values, 1)[0]
        tendencia = 'creciente' if slope > 0.5 else ('decreciente' if slope < -0.5 else 'estable')
    else:
        tendencia = 'sin datos suficientes'
    
    # Estacionalidad (mes con más delitos)
    mes_max = df_mun.groupby('mes')[target].mean().idxmax()
    
    estadisticas_municipio[int(cod_mun)] = {
        'total_historico': int(df_mun[target].sum()),
        'promedio_mensual': round(df_mun[target].mean(), 2),
        'mediana_mensual': round(df_mun[target].median(), 2),
        'maximo_mensual': int(df_mun[target].max()),
        'ultimo_anio': {
            'total': int(total_ultimo),
            'promedio': round(df_mun_ultimo[target].mean(), 2) if len(df_mun_ultimo) > 0 else 0
        },
        'variacion_anual_pct': variacion_pct,
        'tendencia': tendencia,
        'mes_mas_delitos': int(mes_max),
        'poblacion': int(df_mun['poblacion_total'].iloc[-1]) if 'poblacion_total' in df_mun.columns else 0,
        'densidad': round(df_mun['densidad_poblacional'].iloc[-1], 2) if 'densidad_poblacional' in df_mun.columns else 0
    }

print(f"✅ Procesados {len(estadisticas_municipio)} municipios")

# Top 5 municipios con más delitos
print("\n🔴 Top 5 municipios con MÁS delitos (promedio mensual):")
sorted_mun = sorted(estadisticas_municipio.items(), 
                    key=lambda x: x[1]['promedio_mensual'], 
                    reverse=True)[:5]
for cod, data in sorted_mun:
    print(f"   {cod}: {data['promedio_mensual']:.1f} delitos/mes")

# Top 5 municipios con menos delitos
print("\n🟢 Top 5 municipios con MENOS delitos (promedio mensual):")
sorted_mun_asc = sorted(estadisticas_municipio.items(), 
                        key=lambda x: x[1]['promedio_mensual'])[:5]
for cod, data in sorted_mun_asc:
    print(f"   {cod}: {data['promedio_mensual']:.1f} delitos/mes")

 CALCULANDO ESTADÍSTICAS POR MUNICIPIO
✅ Procesados 87 municipios

🔴 Top 5 municipios con MÁS delitos (promedio mensual):
   68001: 1039.2 delitos/mes
   68081: 352.4 delitos/mes
   68276: 296.9 delitos/mes
   68307: 216.9 delitos/mes
   68547: 184.8 delitos/mes

🟢 Top 5 municipios con MENOS delitos (promedio mensual):
   68686: 1.5 delitos/mes
   68160: 1.6 delitos/mes
   68522: 1.7 delitos/mes
   68468: 1.7 delitos/mes
   68245: 1.7 delitos/mes


---
## 3. Tendencias Temporales

In [5]:
# 3.1 Tendencias por año y mes
print(" CALCULANDO TENDENCIAS TEMPORALES")
print("=" * 60)

tendencias_temporales = {
    'por_anio': {},
    'por_mes': {},
    'estacionalidad': {}
}

# Por año
for anio in sorted(df['anio'].unique()):
    df_anio = df[df['anio'] == anio]
    
    tendencias_temporales['por_anio'][int(anio)] = {
        'total_delitos': int(df_anio[target].sum()),
        'promedio_mensual': round(df_anio[target].mean(), 2),
        'municipios_activos': int(df_anio['codigo_municipio'].nunique())
    }

# Por mes (estacionalidad)
meses_nombres = ['Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio',
                 'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre']

for mes in range(1, 13):
    df_mes = df[df['mes'] == mes]
    
    tendencias_temporales['por_mes'][mes] = {
        'nombre': meses_nombres[mes-1],
        'total_historico': int(df_mes[target].sum()),
        'promedio': round(df_mes[target].mean(), 2),
        'desviacion': round(df_mes[target].std(), 2)
    }

# Identificar estacionalidad
promedios_mes = {m: d['promedio'] for m, d in tendencias_temporales['por_mes'].items()}
meses_ordenados = sorted(promedios_mes.items(), key=lambda x: x[1], reverse=True)

tendencias_temporales['estacionalidad'] = {
    'meses_mas_delitos': [
        {'mes': m, 'nombre': meses_nombres[m-1], 'promedio': p} 
        for m, p in meses_ordenados[:3]
    ],
    'meses_menos_delitos': [
        {'mes': m, 'nombre': meses_nombres[m-1], 'promedio': p} 
        for m, p in meses_ordenados[-3:]
    ],
    'variacion_estacional_pct': round(
        (meses_ordenados[0][1] - meses_ordenados[-1][1]) / meses_ordenados[-1][1] * 100, 2
    )
}

print("\n Meses con MÁS delitos (promedio histórico):")
for item in tendencias_temporales['estacionalidad']['meses_mas_delitos']:
    print(f"   {item['nombre']}: {item['promedio']:.1f} delitos/municipio")

print("\n Meses con MENOS delitos (promedio histórico):")
for item in tendencias_temporales['estacionalidad']['meses_menos_delitos']:
    print(f"   {item['nombre']}: {item['promedio']:.1f} delitos/municipio")

print(f"\n Variación estacional: {tendencias_temporales['estacionalidad']['variacion_estacional_pct']:.1f}%")

 CALCULANDO TENDENCIAS TEMPORALES

 Meses con MÁS delitos (promedio histórico):
   Septiembre: 40.8 delitos/municipio
   Marzo: 40.2 delitos/municipio
   Enero: 39.6 delitos/municipio

 Meses con MENOS delitos (promedio histórico):
   Abril: 37.8 delitos/municipio
   Junio: 37.7 delitos/municipio
   Noviembre: 37.1 delitos/municipio

 Variación estacional: 9.7%


---
## 4. Rankings y Comparativas

In [6]:
# 4.1 Rankings y alertas
print(" GENERANDO RANKINGS Y COMPARATIVAS")
print("=" * 60)

comparativas = {
    'rankings': {},
    'alertas': {},
    'contexto_departamental': {}
}

# Ranking por total de delitos (último año)
ranking_total = sorted(
    [(cod, data['ultimo_anio']['total']) for cod, data in estadisticas_municipio.items()],
    key=lambda x: x[1], reverse=True
)
comparativas['rankings']['por_total_ultimo_anio'] = [
    {'posicion': i+1, 'codigo_municipio': cod, 'total_delitos': total}
    for i, (cod, total) in enumerate(ranking_total[:10])
]

# Ranking por promedio mensual
ranking_promedio = sorted(
    [(cod, data['promedio_mensual']) for cod, data in estadisticas_municipio.items()],
    key=lambda x: x[1], reverse=True
)
comparativas['rankings']['por_promedio_mensual'] = [
    {'posicion': i+1, 'codigo_municipio': cod, 'promedio': prom}
    for i, (cod, prom) in enumerate(ranking_promedio[:10])
]

# Municipios más seguros
comparativas['rankings']['mas_seguros'] = [
    {'posicion': i+1, 'codigo_municipio': cod, 'promedio': prom}
    for i, (cod, prom) in enumerate(ranking_promedio[-10:][::-1])
]

# Alertas: municipios con mayor incremento
incrementos = sorted(
    [(cod, data['variacion_anual_pct']) for cod, data in estadisticas_municipio.items()],
    key=lambda x: x[1], reverse=True
)
comparativas['alertas']['mayor_incremento'] = [
    {'codigo_municipio': cod, 'incremento_pct': inc}
    for cod, inc in incrementos[:5] if inc > 0
]

# Alertas: municipios con mayor reducción
comparativas['alertas']['mayor_reduccion'] = [
    {'codigo_municipio': cod, 'reduccion_pct': abs(inc)}
    for cod, inc in incrementos[-5:] if inc < 0
]

# Contexto departamental
promedio_departamental = df[target].mean()
comparativas['contexto_departamental'] = {
    'promedio_mensual': round(promedio_departamental, 2),
    'municipios_sobre_promedio': sum(1 for d in estadisticas_municipio.values() 
                                      if d['promedio_mensual'] > promedio_departamental),
    'municipios_bajo_promedio': sum(1 for d in estadisticas_municipio.values() 
                                     if d['promedio_mensual'] <= promedio_departamental)
}

print("\n Top 5 municipios con MÁS delitos (último año):")
for item in comparativas['rankings']['por_total_ultimo_anio'][:5]:
    print(f"   {item['posicion']}. Municipio {item['codigo_municipio']}: {item['total_delitos']:,} delitos")

print("\n Municipios con MAYOR incremento:")
for item in comparativas['alertas']['mayor_incremento']:
    print(f"   Municipio {item['codigo_municipio']}: +{item['incremento_pct']:.1f}%")

print("\n Municipios con MAYOR reducción:")
for item in comparativas['alertas']['mayor_reduccion']:
    print(f"   Municipio {item['codigo_municipio']}: -{item['reduccion_pct']:.1f}%")

 GENERANDO RANKINGS Y COMPARATIVAS

 Top 5 municipios con MÁS delitos (último año):
   1. Municipio 68001: 15,101 delitos
   2. Municipio 68081: 4,818 delitos
   3. Municipio 68276: 4,137 delitos
   4. Municipio 68307: 3,140 delitos
   5. Municipio 68547: 2,811 delitos

 Municipios con MAYOR incremento:
   Municipio 68684: +418.2%
   Municipio 68368: +350.0%
   Municipio 68522: +300.0%
   Municipio 68867: +190.9%
   Municipio 68673: +175.0%

 Municipios con MAYOR reducción:
   Municipio 68773: -42.4%
   Municipio 68121: -45.5%
   Municipio 68160: -50.0%
   Municipio 68245: -50.0%
   Municipio 68686: -57.1%


---
## 5. Respuestas para Chatbot

In [7]:
# 5.1 Generar respuestas pre-calculadas para chatbot
print(" GENERANDO RESPUESTAS PARA CHATBOT")
print("=" * 60)

# Municipio con más y menos delitos
mun_mas = ranking_promedio[0]
mun_menos = ranking_promedio[-1]

respuestas_chatbot = {
    'estadisticas_clave': {
        'total_delitos_historico': estadisticas_generales['delitos_historico']['total_acumulado'],
        'promedio_mensual_departamento': estadisticas_generales['delitos_historico']['promedio_mensual_global'],
        'total_municipios': estadisticas_generales['cobertura']['municipios'],
        'periodo': f"{estadisticas_generales['periodo']['inicio']}-{estadisticas_generales['periodo']['fin']}"
    },
    'municipio_mas_delitos': {
        'codigo': mun_mas[0],
        'promedio_mensual': mun_mas[1],
        'total_ultimo_anio': estadisticas_municipio[mun_mas[0]]['ultimo_anio']['total']
    },
    'municipio_mas_seguro': {
        'codigo': mun_menos[0],
        'promedio_mensual': mun_menos[1],
        'total_ultimo_anio': estadisticas_municipio[mun_menos[0]]['ultimo_anio']['total']
    },
    'estacionalidad': tendencias_temporales['estacionalidad'],
    'tendencia_departamental': {
        'variacion_ultimo_anio': estadisticas_generales['delitos_ultimo_anio']['variacion_vs_anterior'],
        'direccion': 'incremento' if estadisticas_generales['delitos_ultimo_anio']['variacion_vs_anterior'] > 0 else 'reducción'
    },
    'preguntas_frecuentes': {
        '¿Cuántos delitos hay en promedio al mes?': 
            f"En Santander hay un promedio de {estadisticas_generales['delitos_historico']['promedio_mensual_global']:.1f} delitos por municipio al mes.",
        '¿Cuál es el municipio con más delitos?': 
            f"El municipio {mun_mas[0]} tiene el mayor promedio con {mun_mas[1]:.1f} delitos mensuales.",
        '¿Cuál es el municipio más seguro?': 
            f"El municipio {mun_menos[0]} es el más seguro con solo {mun_menos[1]:.1f} delitos mensuales en promedio.",
        '¿En qué mes hay más delitos?': 
            f"{tendencias_temporales['estacionalidad']['meses_mas_delitos'][0]['nombre']} es el mes con más delitos ({tendencias_temporales['estacionalidad']['meses_mas_delitos'][0]['promedio']:.1f} en promedio).",
        '¿Cómo va la tendencia este año?': 
            f"El último año hubo una {estadisticas_generales['delitos_ultimo_anio']['variacion_vs_anterior']:+.1f}% variación respecto al año anterior.",
        '¿Cuántos municipios están por encima del promedio?':
            f"{comparativas['contexto_departamental']['municipios_sobre_promedio']} municipios están por encima del promedio departamental."
    }
}

print("\n Preguntas frecuentes generadas:")
for pregunta, respuesta in respuestas_chatbot['preguntas_frecuentes'].items():
    print(f"\n   Q: {pregunta}")
    print(f"   A: {respuesta}")

 GENERANDO RESPUESTAS PARA CHATBOT

 Preguntas frecuentes generadas:

   Q: ¿Cuántos delitos hay en promedio al mes?
   A: En Santander hay un promedio de 39.0 delitos por municipio al mes.

   Q: ¿Cuál es el municipio con más delitos?
   A: El municipio 68001 tiene el mayor promedio con 1039.2 delitos mensuales.

   Q: ¿Cuál es el municipio más seguro?
   A: El municipio 68686 es el más seguro con solo 1.5 delitos mensuales en promedio.

   Q: ¿En qué mes hay más delitos?
   A: Septiembre es el mes con más delitos (40.8 en promedio).

   Q: ¿Cómo va la tendencia este año?
   A: El último año hubo una +21.3% variación respecto al año anterior.

   Q: ¿Cuántos municipios están por encima del promedio?
   A: 8 municipios están por encima del promedio departamental.


---
## 6. Guardar Archivos JSON

In [8]:
# 6.1 Guardar todos los archivos JSON
print(" GUARDANDO ARCHIVOS JSON")
print("=" * 60)

archivos = {
    'estadisticas_generales.json': estadisticas_generales,
    'estadisticas_por_municipio.json': estadisticas_municipio,
    'tendencias_temporales.json': tendencias_temporales,
    'comparativas.json': comparativas,
    'respuestas_chatbot.json': respuestas_chatbot
}

for nombre, datos in archivos.items():
    path = OUTPUT_DIR / nombre
    with open(path, 'w', encoding='utf-8') as f:
        json.dump(datos, f, indent=2, ensure_ascii=False)
    print(f"✅ {nombre}")

print(f"\n📁 Archivos guardados en: {OUTPUT_DIR}")

 GUARDANDO ARCHIVOS JSON
✅ estadisticas_generales.json
✅ estadisticas_por_municipio.json
✅ tendencias_temporales.json
✅ comparativas.json
✅ respuestas_chatbot.json

📁 Archivos guardados en: /Users/byverbel/quick_projects/Datos-al-Ecosistema/models/descriptivo/regression_monthly


---
## 7. Ejemplo de Uso

In [9]:
# 7.1 Ejemplo de uso en tablero/chatbot
print(" EJEMPLO DE USO EN TABLERO/CHATBOT")
print("=" * 60)

ejemplo = '''
# En el backend del tablero o chatbot:

import json

# Cargar respuestas pre-generadas
with open('models/descriptivo/regression_monthly/respuestas_chatbot.json', 'r') as f:
    respuestas = json.load(f)

# Responder preguntas instantáneamente
def responder_pregunta(pregunta):
    pregunta_lower = pregunta.lower()
    
    if 'promedio' in pregunta_lower and 'delitos' in pregunta_lower:
        return respuestas['preguntas_frecuentes']['¿Cuántos delitos hay en promedio al mes?']
    
    if 'más delitos' in pregunta_lower or 'mayor' in pregunta_lower:
        return respuestas['preguntas_frecuentes']['¿Cuál es el municipio con más delitos?']
    
    if 'seguro' in pregunta_lower or 'menos delitos' in pregunta_lower:
        return respuestas['preguntas_frecuentes']['¿Cuál es el municipio más seguro?']
    
    if 'mes' in pregunta_lower:
        return respuestas['preguntas_frecuentes']['¿En qué mes hay más delitos?']
    
    if 'tendencia' in pregunta_lower or 'año' in pregunta_lower:
        return respuestas['preguntas_frecuentes']['¿Cómo va la tendencia este año?']
    
    return "Lo siento, no tengo información sobre eso."

# Ejemplos:
print(responder_pregunta("¿Cuál municipio tiene más delitos?"))
print(responder_pregunta("¿Qué mes es más peligroso?"))
print(responder_pregunta("¿Cuál es el municipio más seguro?"))
'''

print(ejemplo)

 EJEMPLO DE USO EN TABLERO/CHATBOT

# En el backend del tablero o chatbot:

import json

# Cargar respuestas pre-generadas
with open('models/descriptivo/regression_monthly/respuestas_chatbot.json', 'r') as f:
    respuestas = json.load(f)

# Responder preguntas instantáneamente
def responder_pregunta(pregunta):
    pregunta_lower = pregunta.lower()

    if 'promedio' in pregunta_lower and 'delitos' in pregunta_lower:
        return respuestas['preguntas_frecuentes']['¿Cuántos delitos hay en promedio al mes?']

    if 'más delitos' in pregunta_lower or 'mayor' in pregunta_lower:
        return respuestas['preguntas_frecuentes']['¿Cuál es el municipio con más delitos?']

    if 'seguro' in pregunta_lower or 'menos delitos' in pregunta_lower:
        return respuestas['preguntas_frecuentes']['¿Cuál es el municipio más seguro?']

    if 'mes' in pregunta_lower:
        return respuestas['preguntas_frecuentes']['¿En qué mes hay más delitos?']

    if 'tendencia' in pregunta_lower or 'año' i

In [10]:
# 7.2 Resumen final
print(" RESUMEN DEL MODELO DESCRIPTIVO")
print("=" * 60)

print(f"""
 ARCHIVOS GENERADOS:
   • estadisticas_generales.json - Métricas globales
   • estadisticas_por_municipio.json - Detalle por municipio
   • tendencias_temporales.json - Patrones anuales y mensuales
   • comparativas.json - Rankings y alertas
   • respuestas_chatbot.json - Respuestas pre-generadas

 ESTADÍSTICAS CLAVE:
   • Período: {estadisticas_generales['periodo']['inicio']}-{estadisticas_generales['periodo']['fin']}
   • Municipios: {estadisticas_generales['cobertura']['municipios']}
   • Total delitos histórico: {estadisticas_generales['delitos_historico']['total_acumulado']:,}
   • Promedio mensual: {estadisticas_generales['delitos_historico']['promedio_mensual_global']:.1f}

 PREGUNTAS QUE PUEDE RESPONDER:
   • ¿Cuántos delitos hay en promedio?
   • ¿Cuál es el municipio más/menos seguro?
   • ¿En qué mes hay más delitos?
   • ¿Cuál es la tendencia actual?
   • ¿Qué municipios mejoraron/empeoraron?

✅ Modelo descriptivo completado exitosamente
📁 Ubicación: {OUTPUT_DIR}
""")

 RESUMEN DEL MODELO DESCRIPTIVO

 ARCHIVOS GENERADOS:
   • estadisticas_generales.json - Métricas globales
   • estadisticas_por_municipio.json - Detalle por municipio
   • tendencias_temporales.json - Patrones anuales y mensuales
   • comparativas.json - Rankings y alertas
   • respuestas_chatbot.json - Respuestas pre-generadas

 ESTADÍSTICAS CLAVE:
   • Período: 2010-2025
   • Municipios: 87
   • Total delitos histórico: 508,227
   • Promedio mensual: 39.0

 PREGUNTAS QUE PUEDE RESPONDER:
   • ¿Cuántos delitos hay en promedio?
   • ¿Cuál es el municipio más/menos seguro?
   • ¿En qué mes hay más delitos?
   • ¿Cuál es la tendencia actual?
   • ¿Qué municipios mejoraron/empeoraron?

✅ Modelo descriptivo completado exitosamente
📁 Ubicación: /Users/byverbel/quick_projects/Datos-al-Ecosistema/models/descriptivo/regression_monthly

